### 4.1.1 Parse the fasta file (1 point)

How many sequences are contained in the file protein_N_data.fasta?
List the names of the sequences.

In [23]:
from Bio import SeqIO
from pathlib import Path

# pwd = Path.absolute()

records = SeqIO.parse(f"/home/henrik/BINF200_H23/assignments/data2/Coronavirus/Coronavirus/protein_N_data.fasta", "fasta")

#Names:
print(f" Names of sequences : {[r.name for r in records]} ")

records = SeqIO.parse(f"/home/henrik/BINF200_H23/assignments/data2/Coronavirus/Coronavirus/protein_N_data.fasta", "fasta")

#Lengths:
print(f" Length of induvidual sequences : {[len(r.seq) for r in records]} ")

records = SeqIO.parse(f"/home/henrik/BINF200_H23/assignments/data2/Coronavirus/Coronavirus/protein_N_data.fasta", "fasta")

#Amount of sequences:

records = SeqIO.parse(f"/home/henrik/BINF200_H23/assignments/data2/Coronavirus/Coronavirus/protein_N_data.fasta", "fasta")

length = len([r for r in records])
print(f" Length of sequences: {length}")



 Names of sequences : ['Human_NL63_alpha', 'Bat_alpha', 'mink_alpha', 'camel_alpha', 'ferret_alpha', 'rat_alpha', 'rabbit_beta', 'MERS_beta', 'HKU24_beta', 'England1_beta', 'SARS_beta', 'whale_gamma', 'turkey_gamma', 'duck_gamma', 'goose_gamma', 'sparrow_delta', 'wigeon_delta', 'nightHeron_delta', 'moorhen_delta', 'porcine_delta'] 
 Length of induvidual sequences : [1134, 1278, 1131, 1149, 1128, 1176, 1335, 1242, 1332, 1236, 1260, 1140, 1230, 1245, 1245, 1029, 1053, 1029, 1056, 1029] 
 Length of sequences: 20


### 4.1.2 Find protein N in a specific coronavirus genome (1 point)

From the sequences in protein_N_data.fasta, find the sequence for which the first letter
in its name is closest in the alphabet to the first letter in your first name. If there
are multiple sequences starting with the same letter, pick one arbitrarily. For the selected
sequence:

**Human_NL63_alpha**

• Find the assembly accession ID in the table above.
• Go to the NCBI website (cf. links above) and find the corresponding genome assembly.
• What are the genomic coordinates (start and end position) of gene N in this genome?
(Hint: follow the RefSeq link)

Link to the gene ["gene"]("https://www.ncbi.nlm.nih.gov/gene/2943504/")

The genes range is : ***26133 - 27266*** 

### 4.1.3 Multiple sequence alignment (1 point)

Build a multiple sequence alignment for the protein_N_data using a multiple sequence
alignment tool of your choice. (Hint: check out the services provided by EMBL’s European
Bioinformatics Institute (EMBL-EBI).)

For this question i used:

https://www.bioinformatics.nl/cgi-bin/emboss/emma

and got out a file, containing aligned sequences.

### 4.1.4 Phylogenetic tree reconstruction (1 point)

Based on the results from the previous step, build a phylogenetic tree. (Hint: at this stage
it is not required to make an “advanced tree”, providing a simple tree is enough). Save the
image of the phylogenetic/guide tree.


Tree was made in this link:

https://www.ebi.ac.uk/Tools/services/web/toolresult.ebi?jobId=simple_phylogeny-I20231002-105856-0957-35098453-p1m

From the MSL

Image:

![tree](outdata2/treeout.png)



### 4.1.5 Interpretation (1 point)

Based on the results from the previous two steps, what do you see? Elaborate with a small
text (3-4 lines): Explain what you observe from the multiple sequence alignment itself (hint:
check the number of conserved sites), and give a short interpretation of the phylogenetic
tree you have constructed.

From the tree i can see that virus variants are genetically grouped together by their "variants"
